<a href="https://colab.research.google.com/github/xorud0771/booklist/blob/main/%EC%BD%94%EB%9E%A9%ED%85%8C%EC%8A%A4%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print('HI')

HI


In [ ]:
%time

import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 8.58 µs


In [ ]:
%%time

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cifar10_train = CIFAR10("./data", train=True, download=True, transform=transforms.transforms.ToTensor())
cifar10_test = CIFAR10("./data", train=False, download=True, transform=transforms.transforms.ToTensor())

batch_size = 64
train_loader = DataLoader(cifar10_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(cifar10_test, batch_size=len(cifar10_test), shuffle=False)

class Net(nn.Module):
  def __init__(self): # __init__ 함수에 self 인자 추가
    super(Net, self).__init__() # super() 함수를 사용하여 부모 클래스 초기화
    self.conv1 = nn.Conv2d(3,6,5)
    self.pool = nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(6,16,5)
    self.fc1 = nn.Linear(16*5*5, 256)
    self.fc2 = nn.Linear(256,10)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16*5*5)
    x = F.relu(self.fc1(x)) # self.fc1을 통해 형태 변환 및 활성화 함수 적용
    x = self.fc2(x) # self.fc2(x)로 수정
    return x
net = Net()
if torch.cuda.is_available():
  net.cuda()

loss_fnc = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

record_loss_train = []
record_loss_test = []

x_test, t_test = next(iter(test_loader)) # t_test로 변수명 수정
x_test, t_test = x_test.to(device), t_test.to(device) # t_test로 변수명 수정

for i in range(10):
  net.train()
  loss_train = 0.0
  for j, (x,t) in enumerate(train_loader):
    x_train, y_train = x.to(device), t.to(device) # x_train, y_train 변수에 x, t 할당
    y=net(x_train)
    loss = loss_fnc(y,y_train)
    loss_train =loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  loss_train /=(j+1)
  record_loss_train.append(loss_train)

  net.eval()
  y_test = net(x_test)
  loss_test = loss_fnc(y_test,t_test).item() # t_test로 변수명 수정
  record_loss_test.append(loss_test)

# if torch.cuda.is_available(): # for 루프 밖으로 이동
#   x,t = x.cuda(), t.cuda( ) # for 루프 밖으로 이동, 필요 없는 코드 삭제

cpu
CPU times: user 4min 5s, sys: 8.11 s, total: 4min 13s
Wall time: 4min 15s
